In [18]:
from glob import glob
import matplotlib
import pandas as pd
import numpy as np
import os
import matplotlib.pylab as plt
from matplotlib import cm
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils import shuffle

matplotlib.style.use('ggplot')
%matplotlib inline

In [53]:
Y_class = {'Brush_teeth':0, 
              'Use_telephone':1, 
              'Drink_glass':2, 
              'Comb_hair':3, 
              'Walk':4,
              'Standup_chair':5,
              'Sitdown_chair':6,
              'Pour_water':7,
              'Getup_bed':8,
              'Liedown_bed':9,
              'Eat_meat':10,
              'Descend_stairs':11
             }

df = []
data = None
def loadDataset():
    global Y_class
    global data
    size = 32
    data = pd.DataFrame()
    for c in Y_class:
        f_list = glob("./HMP_Dataset/" + c + "/*")
        df = pd.DataFrame()
        for f in f_list:
            df_file = pd.read_table(f, sep=' ', header=None, names=['x', 'y', 'z'])
            for i in range(0, len(df_file) - size, size):
                chunk = df_file.iloc[i:i+32].values.flatten()
                df = df.append([np.append(chunk,Y_class[c])])

        data = data.append(df)
    
loadDataset()
display(data)

,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,16,45,34,16,45,33,16,44,35,16,...,27,50,40,30,51,43,32,48,41,0
0,32,50,41,32,49,42,32,50,42,33,...,27,59,38,26,55,42,27,53,39,0
0,28,60,38,25,52,42,28,53,37,26,...,34,50,44,33,56,50,34,44,41,0
0,27,43,56,35,56,54,34,42,42,33,...,36,42,46,37,38,49,32,38,50,0
0,31,47,52,31,45,55,29,45,50,27,...,24,53,40,27,50,39,31,51,41,0
0,32,49,42,34,48,41,35,47,42,36,...,25,51,39,24,51,39,25,52,39,0
0,27,52,39,25,54,42,26,51,40,25,...,31,55,45,33,49,43,33,51,37,0
0,37,56,39,39,51,39,41,51,35,39,...,37,48,37,36,51,37,37,54,40,0
0,36,51,38,36,53,39,40,55,43,37,...,36,49,43,36,49,43,36,49,45,0
0,36,48,44,36,49,44,36,51,45,37,...,36,47,46,35,46,45,35,45,48,0
